In [1]:
import pandas as pd 
import os
import cv2
import fnmatch
%matplotlib inline
import matplotlib.image as mpimg
import keras
from keras import backend as K
from keras.models import Sequential,load_model,Model
from keras.losses import binary_crossentropy
from keras.optimizers import Adam,RMSprop,Nadam
from keras.layers import Dense, Activation, Dropout, Flatten,Conv2D, MaxPooling2D, GlobalAveragePooling2D, Input,SeparableConv2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau,TensorBoard
from keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
from keras.applications import VGG19,VGG16,ResNet50,Xception
import numpy as np
from keras.utils import np_utils
from sklearn.utils import shuffle
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
dict={7:"ok",6:"index",8:"palm_moved",0:"down",3:"fist",9:"c",2:"l",5:"thumb",4:"fist_moved",1:"palm"}

Using TensorFlow backend.


In [2]:
def extract_x(d1):
    x = []
    for d2 in os.listdir(path+d1):
        for root,dirs,files in os.walk(path+d1+'/'+d2):
            for f in files:
                if fnmatch.fnmatch(f,"*.png"):
                    img = cv2.imread(path+d1+'/'+d2+'/'+f)
                    img = cv2.resize(img,(224,224))
                    x.append(img)
    return x

In [3]:
def extract_y(d1):
    y = []
    y1 = [] 
    for d2 in os.listdir(path+d1):
        for root,dirs,files in os.walk(path+d1+'/'+d2):
            for f in files:
                y.append(d2[1])
    encoder = LabelEncoder()
    encoder.fit(y)
    encoded_Y = encoder.transform(y)
    # convert integers to dummy variables (i.e. one hot encoded)
    y1 = np_utils.to_categorical(encoded_Y)
    return y1

In [4]:
def model():
    # (3) Create a sequential model
    model = Sequential()

    # 1st Convolutional Layer
    model.add(Conv2D(filters=96, input_shape=(224,224,3), kernel_size=(11,11),strides=(4,4), padding='valid'))
    model.add(Activation('relu'))
    # Pooling 
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
    # Batch Normalisation before passing it to the next layer
    model.add(BatchNormalization())

    # 2nd Convolutional Layer
    model.add(Conv2D(filters=256, kernel_size=(11,11), strides=(1,1), padding='valid'))
    model.add(Activation('relu'))
    # Pooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
    # Batch Normalisation
    model.add(BatchNormalization())

    # 3rd Convolutional Layer
    model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
    model.add(Activation('relu'))
    # Batch Normalisation
    model.add(BatchNormalization())

    # 4th Convolutional Layer
    model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1), padding='valid'))
    model.add(Activation('relu'))
    # Batch Normalisation
    model.add(BatchNormalization())

    # 5th Convolutional Layer
    model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1), padding='valid'))
    model.add(Activation('relu'))
    # Pooling
    model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2), padding='valid'))
    # Batch Normalisation
    model.add(BatchNormalization())

    # Passing it to a dense layer
    model.add(Flatten())
    # 1st Dense Layer
    model.add(Dense(4096, input_shape=(224*224*3,)))
    model.add(Activation('relu'))
    # Add Dropout to prevent overfitting
    model.add(Dropout(0.4))
    # Batch Normalisation
    model.add(BatchNormalization())

    # 2nd Dense Layer
    model.add(Dense(4096))
    model.add(Activation('relu'))
    # Add Dropout
    model.add(Dropout(0.4))
    # Batch Normalisation
    model.add(BatchNormalization())

    # 3rd Dense Layer
    model.add(Dense(1000))
    model.add(Activation('relu'))
    # Add Dropout
    model.add(Dropout(0.4))
    # Batch Normalisation
    model.add(BatchNormalization())

    # Output Layer
    model.add(Dense(10))
    model.add(Activation('softmax'))

    # (4) Compile 
    model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])
    filepath = "RPS.h5"
    checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
    callbacks_list = [checkpoint]
    return model, callbacks_list

In [5]:
path = "../input/leapgestrecog/leapGestRecog/"
model, callbacks_list = model()
dir = os.listdir(path)
for d in dir:
    x = []
    y = []
    x = extract_x(d)
    y = extract_y(d)
    x,y = shuffle(x,y,random_state=0)
    lb=(int)(0.6*len(y))
    train_x,valid_x = np.array(x[:lb]),np.array(x[lb:])
    train_y,valid_y = np.array(y[:lb]),np.array(y[lb:])
    history = model.fit(train_x, train_y, validation_data=(valid_x, valid_y), epochs=2, batch_size=32
              ,callbacks=callbacks_list)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Train on 1200 samples, validate on 800 samples
Epoch 1/2
1200/1200 [==============================] - 7s 6ms/step - loss: 0.7862 - acc: 0.7433 - val_loss: 13.7908 - val_acc: 0.1100

Epoch 00001: val_acc improved from -inf to 0.11000, saving model to RPS.h5
Epoch 2/2
1200/1200 [==============================] - 2s 2ms/step - loss: 0.0912 - acc: 0.9725 - val_loss: 1.2779 - val_acc: 0.7438

Epoch 00002: val_acc improved from 0.11000 to 0.74375, saving model to RPS.h5
Train on 1200 samples, validate on 800 samples
Epoch 1/2
1200/1200 [==============================] - 2s 2ms/step - loss: 0.5590 - acc: 0.8767 - val_loss: 6.3695 - val_acc: 0.3900

Epoch 00001: val_acc did not improve from 0.74375
Epoch 2/2
1200/1200 [==============================] - 2s 2ms/ste

In [6]:
loss,mse = model.evaluate(x=valid_x, y=valid_y, batch_size=64, verbose=1, sample_weight=None, steps=None)
print("Loss:{:.3f}".format(loss))

800/800 [==============================] - 1s 1ms/step
Loss:0.000
